### Sql queries

Prerequsites - sudo apt-get install python-mysqldb

In [51]:
import MySQLdb
import ConfigParser
configParser = ConfigParser.ConfigParser()
configParser.readfp(open("DbConfig.txt"))
host = configParser.get("dbconfig", "host")
user = configParser.get("dbconfig", "user")
userpw = configParser.get("dbconfig", "userpw")
db = configParser.get("dbconfig", "db")


In [52]:
db = MySQLdb.connect(host = host, user = user, passwd = userpw, db = db)
cursor = db.cursor()

sql = (""""
SELECT vehicle_type.vehicle_type, count( a.accident_index )
FROM accidents a, vehicles v, police_force, vehicle_type
WHERE v.acc_index
IN (

SELECT a.accident_index
FROM accidents a, vehicles v, police_force, vehicle_type
WHERE a.accident_index = v.acc_index
AND v.vehicle_type = vehicle_type.code
AND v.vehicle_type =1
)
AND a.accident_index = v.acc_index
AND v.vehicle_type = vehicle_type.code
AND a.accident_severity
IN ( 1, 2 )
AND a.police_force = police_force.code
AND police_force.label
IN (
"West Midlands"
)
AND year( a.date )
IN ( 2010, 2011, 2012, 2013, 2014 )
GROUP BY 1
LIMIT 0 , 30;
"""
)

try:
    cursor.execute(sql)
    results = cursor.fetcall()
    for row in results:
        print row
except Exception as e:
    print "Error; query failed"
    print e

db.close()

Error; query failed
(1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'"\nSELECT vehicle_type.vehicle_type, count( a.accident_index )\nFROM accidents a, \' at line 1')
